In [37]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("data/notes.tsv", sep="\t")
print(data.shape)

C:\Users\Chico\AppData\Local\Temp\ipykernel_11576\3498016289.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/notes.tsv", sep="\t")


(1537510, 23)


## Creating data subset for annotation

In [41]:
annoNotes = pd.DataFrame(columns=["noteId", "classification", "summary"])
misCount = 0
nMisCount = 0
index = 0

for i in range(1000):

    noteId = data.iloc[i]["noteId"]
    classification = data.iloc[i]["classification"]
    believable = data.iloc[i]["believable"]
    harmful = data.iloc[i]["harmful"]
    valDiff = data.iloc[i]["validationDifficulty"]
    summary = data.iloc[i]["summary"]

    if (classification == "MISINFORMED_OR_POTENTIALLY_MISLEADING" and 
        believable not in ["BELIEVABLEBYFEW", "BELIEVABLE_BY_MANY"] and
        harmful not in ["LITTLE*HARM", "CONSIDERABLE*HARM"] and 
        valDiff not in ["EASY", "CHALLENGING"] and
        "http" in summary):
        misCount += 1
        annoNotes.loc[index, "noteId"] = noteId
        annoNotes.loc[index, "summary"] = summary
        index += 1

    if index >= 201:
        break


annoNotes.to_csv("data/annotatation.csv")